In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import seaborn as sns
py.init_notebook_mode(connected=True)
%matplotlib inline

Using TensorFlow backend.


In [2]:
import quandl
data = quandl.get('BCHARTS/KRAKENUSD', returns='pandas')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1523 entries, 2014-01-07 to 2018-03-09
Data columns (total 7 columns):
Open                 1523 non-null float64
High                 1523 non-null float64
Low                  1523 non-null float64
Close                1523 non-null float64
Volume (BTC)         1523 non-null float64
Volume (Currency)    1523 non-null float64
Weighted Price       1523 non-null float64
dtypes: float64(7)
memory usage: 95.2 KB


In [4]:
data.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


In [5]:
data.tail()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2018-03-05,11503.1,11666.4,11400.8,11433.9,4954.252606,5.710797e+07,11527.060159
2018-03-06,11432.9,11438.0,10550.1,10731.5,10018.322458,1.096734e+08,10947.286841
2018-03-07,10718.4,10899.0,9388.1,9903.0,14472.601157,1.465536e+08,10126.276122
2018-03-08,9903.0,10120.7,9000.0,9279.9,11725.829587,1.123067e+08,9577.715294
2018-03-09,9290.1,9427.9,8400.0,9209.0,18786.104345,1.666896e+08,8873.026705


In [6]:
btc_trace = go.Scatter(x=data.index, y=data['Weighted Price'], name= 'Price')
py.iplot([btc_trace])

In [7]:
data['Weighted Price'].replace(0, np.nan, inplace=True)
data['Weighted Price'].fillna(method='ffill', inplace=True)

In [8]:
btc_trace = go.Scatter(x=data.index, y=data['Weighted Price'], name= 'Price')
py.iplot([btc_trace])

In [9]:
from sklearn.preprocessing import MinMaxScaler
values = data['Weighted Price'].values.reshape(-1,1)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [10]:
values

array([[   841.83551025],
       [   839.15625   ],
       [   831.57293701],
       ..., 
       [ 10126.27636719],
       [  9577.71484375],
       [  8873.02636719]], dtype=float32)

In [11]:
train_size = int(len(scaled) * 0.7)
test_size = len(scaled) - train_size
train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
print(len(train), len(test))

1066 457


In [ ]:
def create_dataset_parts(dataset, proportion=0.7):
    train_size = int(len(scaled) * proportion)
    test_size = len(scaled) - train_size
    train, test = [], []
    sm1 = 0
    for i in range(len(dataset) - 1):
        sm1 += proportion
        a = dataset[i:(i + 1)]
        if sm1 >= 1:
            train.append(a)
            sm1-=1
        else:
            test.append(a)
    return np.array(train), np.array(test)

In [13]:
train, test = create_dataset_parts(scaled)
print(len(train), len(test))

1065 457


In [14]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [15]:
look_back = 300
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

765
157


In [16]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [17]:
model = Sequential()
model.add(LSTM(500, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
history = model.fit(trainX, trainY, epochs=4000, batch_size=200, validation_data=(testX, testY), verbose=1, shuffle=True)

Train on 765 samples, validate on 157 samples
Epoch 1/4000
765/765 [==============================] - 2s 3ms/step - loss: 0.0722 - val_loss: 0.0782
Epoch 2/4000
765/765 [==============================] - 0s 49us/step - loss: 0.0442 - val_loss: 0.1452
Epoch 3/4000
765/765 [==============================] - 0s 43us/step - loss: 0.0396 - val_loss: 0.0853
Epoch 4/4000
765/765 [==============================] - 0s 61us/step - loss: 0.0321 - val_loss: 0.0723
Epoch 5/4000
765/765 [==============================] - 0s 47us/step - loss: 0.0245 - val_loss: 0.0915
Epoch 6/4000
765/765 [==============================] - 0s 41us/step - loss: 0.0230 - val_loss: 0.0581
Epoch 7/4000
765/765 [==============================] - 0s 62us/step - loss: 0.0194 - val_loss: 0.0695
Epoch 8/4000
765/765 [==============================] - 0s 27us/step - loss: 0.0187 - val_loss: 0.0537
Epoch 9/4000
765/765 [==============================] - 0s 61us/step - loss: 0.0181 - val_loss: 0.0558
Epoch 10/4000
765/765 [=====

765/765 [==============================] - 0s 46us/step - loss: 0.0063 - val_loss: 0.0305
Epoch 158/4000
765/765 [==============================] - 0s 56us/step - loss: 0.0068 - val_loss: 0.0298
Epoch 159/4000
765/765 [==============================] - 0s 49us/step - loss: 0.0079 - val_loss: 0.0336
Epoch 160/4000
765/765 [==============================] - 0s 73us/step - loss: 0.0064 - val_loss: 0.0354
Epoch 161/4000
765/765 [==============================] - 0s 44us/step - loss: 0.0064 - val_loss: 0.0361
Epoch 162/4000
765/765 [==============================] - 0s 47us/step - loss: 0.0069 - val_loss: 0.0337
Epoch 163/4000
765/765 [==============================] - 0s 47us/step - loss: 0.0060 - val_loss: 0.0341
Epoch 164/4000
765/765 [==============================] - 0s 37us/step - loss: 0.0065 - val_loss: 0.0350
Epoch 165/4000
765/765 [==============================] - 0s 52us/step - loss: 0.0063 - val_loss: 0.0356
Epoch 166/4000
765/765 [==============================] - 0s 62us/step

765/765 [==============================] - 0s 44us/step - loss: 0.0046 - val_loss: 0.0315
Epoch 314/4000
765/765 [==============================] - 0s 65us/step - loss: 0.0046 - val_loss: 0.0331
Epoch 315/4000
765/765 [==============================] - 0s 50us/step - loss: 0.0049 - val_loss: 0.0322
Epoch 316/4000
765/765 [==============================] - 0s 40us/step - loss: 0.0046 - val_loss: 0.0310
Epoch 317/4000
765/765 [==============================] - 0s 53us/step - loss: 0.0046 - val_loss: 0.0313
Epoch 318/4000
765/765 [==============================] - 0s 68us/step - loss: 0.0046 - val_loss: 0.0316
Epoch 319/4000
765/765 [==============================] - 0s 40us/step - loss: 0.0046 - val_loss: 0.0345
Epoch 320/4000
765/765 [==============================] - 0s 51us/step - loss: 0.0049 - val_loss: 0.0315
Epoch 321/4000
765/765 [==============================] - 0s 48us/step - loss: 0.0044 - val_loss: 0.0320
Epoch 322/4000
765/765 [==============================] - 0s 52us/step

765/765 [==============================] - 0s 43us/step - loss: 0.0044 - val_loss: 0.0334
Epoch 470/4000
765/765 [==============================] - 0s 45us/step - loss: 0.0043 - val_loss: 0.0306
Epoch 471/4000
765/765 [==============================] - 0s 44us/step - loss: 0.0046 - val_loss: 0.0308
Epoch 472/4000
765/765 [==============================] - 0s 64us/step - loss: 0.0048 - val_loss: 0.0331
Epoch 473/4000
765/765 [==============================] - 0s 56us/step - loss: 0.0044 - val_loss: 0.0301
Epoch 474/4000
765/765 [==============================] - 0s 40us/step - loss: 0.0050 - val_loss: 0.0319
Epoch 475/4000
765/765 [==============================] - 0s 55us/step - loss: 0.0057 - val_loss: 0.0332
Epoch 476/4000
765/765 [==============================] - 0s 46us/step - loss: 0.0057 - val_loss: 0.0323
Epoch 477/4000
765/765 [==============================] - 0s 47us/step - loss: 0.0047 - val_loss: 0.0328
Epoch 478/4000
765/765 [==============================] - 0s 46us/step

765/765 [==============================] - 0s 65us/step - loss: 0.0052 - val_loss: 0.0321
Epoch 626/4000
765/765 [==============================] - 0s 45us/step - loss: 0.0059 - val_loss: 0.0325
Epoch 627/4000
765/765 [==============================] - 0s 50us/step - loss: 0.0047 - val_loss: 0.0383
Epoch 628/4000
765/765 [==============================] - 0s 45us/step - loss: 0.0047 - val_loss: 0.0403
Epoch 629/4000
765/765 [==============================] - 0s 55us/step - loss: 0.0050 - val_loss: 0.0415
Epoch 630/4000
765/765 [==============================] - 0s 31us/step - loss: 0.0057 - val_loss: 0.0376
Epoch 631/4000
765/765 [==============================] - 0s 45us/step - loss: 0.0048 - val_loss: 0.0362
Epoch 632/4000
765/765 [==============================] - 0s 43us/step - loss: 0.0042 - val_loss: 0.0375
Epoch 633/4000
765/765 [==============================] - 0s 58us/step - loss: 0.0042 - val_loss: 0.0376
Epoch 634/4000
765/765 [==============================] - 0s 57us/step

...

765/765 [==============================] - 0s 54us/step - loss: 0.0032 - val_loss: 0.0684
Epoch 3326/4000
765/765 [==============================] - 0s 52us/step - loss: 0.0026 - val_loss: 0.0670
Epoch 3327/4000
765/765 [==============================] - 0s 38us/step - loss: 0.0024 - val_loss: 0.0666
Epoch 3328/4000
765/765 [==============================] - 0s 62us/step - loss: 0.0024 - val_loss: 0.0642
Epoch 3329/4000
765/765 [==============================] - 0s 43us/step - loss: 0.0030 - val_loss: 0.0653
Epoch 3330/4000
765/765 [==============================] - 0s 43us/step - loss: 0.0025 - val_loss: 0.0661
Epoch 3331/4000
765/765 [==============================] - 0s 44us/step - loss: 0.0026 - val_loss: 0.0695
Epoch 3332/4000
765/765 [==============================] - 0s 42us/step - loss: 0.0027 - val_loss: 0.0701
Epoch 3333/4000
765/765 [==============================] - 0s 44us/step - loss: 0.0029 - val_loss: 0.0685
Epoch 3334/4000
765/765 [==============================] - 0s 

765/765 [==============================] - 0s 55us/step - loss: 0.0029 - val_loss: 0.0699
Epoch 3480/4000
765/765 [==============================] - 0s 65us/step - loss: 0.0026 - val_loss: 0.0711
Epoch 3481/4000
765/765 [==============================] - 0s 50us/step - loss: 0.0027 - val_loss: 0.0709
Epoch 3482/4000
765/765 [==============================] - 0s 42us/step - loss: 0.0027 - val_loss: 0.0676
Epoch 3483/4000
765/765 [==============================] - 0s 48us/step - loss: 0.0025 - val_loss: 0.0668
Epoch 3484/4000
765/765 [==============================] - 0s 41us/step - loss: 0.0027 - val_loss: 0.0647
Epoch 3485/4000
765/765 [==============================] - 0s 61us/step - loss: 0.0031 - val_loss: 0.0636
Epoch 3486/4000
765/765 [==============================] - 0s 67us/step - loss: 0.0037 - val_loss: 0.0732
Epoch 3487/4000
765/765 [==============================] - 0s 43us/step - loss: 0.0034 - val_loss: 0.0721
Epoch 3488/4000
765/765 [==============================] - 0s 

765/765 [==============================] - 0s 50us/step - loss: 0.0029 - val_loss: 0.0671
Epoch 3634/4000
765/765 [==============================] - 0s 51us/step - loss: 0.0024 - val_loss: 0.0713
Epoch 3635/4000
765/765 [==============================] - 0s 47us/step - loss: 0.0023 - val_loss: 0.0691
Epoch 3636/4000
765/765 [==============================] - 0s 54us/step - loss: 0.0021 - val_loss: 0.0674
Epoch 3637/4000
765/765 [==============================] - 0s 31us/step - loss: 0.0022 - val_loss: 0.0681
Epoch 3638/4000
765/765 [==============================] - 0s 67us/step - loss: 0.0020 - val_loss: 0.0664
Epoch 3639/4000
765/765 [==============================] - 0s 35us/step - loss: 0.0024 - val_loss: 0.0683
Epoch 3640/4000
765/765 [==============================] - 0s 43us/step - loss: 0.0021 - val_loss: 0.0682
Epoch 3641/4000
765/765 [==============================] - 0s 50us/step - loss: 0.0021 - val_loss: 0.0695
Epoch 3642/4000
765/765 [==============================] - 0s 

765/765 [==============================] - 0s 65us/step - loss: 0.0025 - val_loss: 0.0713
Epoch 3788/4000
765/765 [==============================] - 0s 46us/step - loss: 0.0023 - val_loss: 0.0714
Epoch 3789/4000
765/765 [==============================] - 0s 44us/step - loss: 0.0020 - val_loss: 0.0724
Epoch 3790/4000
765/765 [==============================] - 0s 55us/step - loss: 0.0019 - val_loss: 0.0710
Epoch 3791/4000
765/765 [==============================] - 0s 65us/step - loss: 0.0021 - val_loss: 0.0713
Epoch 3792/4000
765/765 [==============================] - 0s 42us/step - loss: 0.0021 - val_loss: 0.0713
Epoch 3793/4000
765/765 [==============================] - 0s 46us/step - loss: 0.0021 - val_loss: 0.0706
Epoch 3794/4000
765/765 [==============================] - 0s 73us/step - loss: 0.0020 - val_loss: 0.0687
Epoch 3795/4000
765/765 [==============================] - 0s 34us/step - loss: 0.0021 - val_loss: 0.0693
Epoch 3796/4000
765/765 [==============================] - 0s 

765/765 [==============================] - 0s 66us/step - loss: 0.0023 - val_loss: 0.0724
Epoch 3942/4000
765/765 [==============================] - 0s 44us/step - loss: 0.0023 - val_loss: 0.0734
Epoch 3943/4000
765/765 [==============================] - 0s 85us/step - loss: 0.0023 - val_loss: 0.0722
Epoch 3944/4000
765/765 [==============================] - 0s 37us/step - loss: 0.0022 - val_loss: 0.0718
Epoch 3945/4000
765/765 [==============================] - 0s 50us/step - loss: 0.0021 - val_loss: 0.0725
Epoch 3946/4000
765/765 [==============================] - 0s 60us/step - loss: 0.0020 - val_loss: 0.0732
Epoch 3947/4000
765/765 [==============================] - 0s 43us/step - loss: 0.0020 - val_loss: 0.0720
Epoch 3948/4000
765/765 [==============================] - 0s 71us/step - loss: 0.0023 - val_loss: 0.0714
Epoch 3949/4000
765/765 [==============================] - 0s 66us/step - loss: 0.0019 - val_loss: 0.0710
Epoch 3950/4000
765/765 [==============================] - 0s 

In [ ]:
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
yhat = model.predict(testX)
pyplot.plot(yhat, label='predict')
pyplot.plot(testY, label='true')
pyplot.legend()
pyplot.show()

In [ ]:
yhat_inverse = scaler.inverse_transform(yhat.reshape(-1, 1))
testY_inverse = scaler.inverse_transform(testY.reshape(-1, 1))

In [ ]:
rmse = sqrt(mean_squared_error(testY_inverse, yhat_inverse))
print('Test RMSE: %.3f' % rmse)

In [ ]:
pyplot.plot(yhat_inverse, label='predict')
pyplot.plot(testY_inverse, label='actual', alpha=0.5)
pyplot.legend()
pyplot.show()

In [ ]:
predictDates = data.tail(len(testX)).index

In [ ]:
testY_reshape = testY_inverse.reshape(len(testY_inverse))
yhat_reshape = yhat_inverse.reshape(len(yhat_inverse))

In [ ]:
actual_chart = go.Scatter(x=predictDates, y=testY_reshape, name= 'Actual Price')
predict_chart = go.Scatter(x=predictDates, y=yhat_reshape, name= 'Predict Price')
py.iplot([predict_chart, actual_chart])

In [ ]:
inpx = trainX[len(trainX) - 1:len(trainX)]

In [ ]:
n = inpx[0,0,1:len(inpx[0,0])]

In [ ]:
index = []
prediction = []
nxt = inpx
index.append([0])

#prediction.append(nxt[0,0].tolist())


In [ ]:
for i in range(1, 100):
    n = nxt[0,0, 1:len(inpx[0,0])]
    predict = model.predict(nxt, batch_size=1, verbose=1)
    n = np.append(n, [predict])
    #print(predict)
    nxt = np.reshape(n, (nxt.shape[0], nxt.shape[1], nxt.shape[2]))
    #print(nxt[0,0].tolist())
    index.append(i)
    prediction.append(predict[0,0].tolist())

In [ ]:
#yhat = model.predict(testX)
#pyplot.plot(yhat, label='predict')
pyplot.plot(prediction, label='predict_history')
pyplot.legend()
pyplot.show()

In [ ]:
prediction_inverse = scaler.inverse_transform(np.reshape(prediction, (99,1)))

In [ ]:
pyplot.plot(prediction_inverse, label='predict_history')
pyplot.legend()
pyplot.show()